In [1]:
import tweepy
from kafka import KafkaProducer
import json
import config.config as config

BEARER_TOKEN = config.BEARER_TOKEN
ALL_EXPANSIONS = ["author_id", "referenced_tweets.id", "edit_history_tweet_ids", "in_reply_to_user_id", "attachments.media_keys", "attachments.poll_ids", "geo.place_id", "entities.mentions.username", "referenced_tweets.id.author_id",]
ALL_MEDIA_FIELDS = ["media_key", "type", "url", "duration_ms", "height", "preview_image_url", "public_metrics", "width", "alt_text", "variants",]
ALL_PLACE_FIELDS = ["full_name", "id", "contained_within", "country", "country_code", "geo", "name", "place_type",]
ALL_POLL_FIELDS = ["id", "options", "duration_minutes", "end_datetime", "voting_status",]
ALL_TWEET_FIELDS = ["id", "text", "attachments", "author_id", "context_annotations", "conversation_id", "created_at", "entities", "geo", "in_reply_to_user_id", "lang", "possibly_sensitive", "public_metrics", "referenced_tweets", "reply_settings", "source", "withheld",]
ALL_USER_FIELDS = ["id", "name", "username", "created_at", "description", "entities", "location", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "verified", "withheld",]

client = tweepy.StreamingClient(BEARER_TOKEN, wait_on_rate_limit=True)

In [4]:
client.add_rules(tweepy.StreamRule("(Anies OR @aniesbaswedan) lang:in -is:retweet", tag="Anies Baswedan;tweets"))
client.add_rules(tweepy.StreamRule("(Ganjar OR @ganjarpranowo) lang:in -is:retweet", tag="Ganjar Pranowo;tweets"))
client.add_rules(tweepy.StreamRule("(Prabowo OR @prabowo) lang:in -is:retweet", tag="Prabowo Subianto;tweets"))
client.add_rules(tweepy.StreamRule("(Puan OR @puanmaharani_ri) lang:in -is:retweet", tag="Puan Maharani;tweets"))
client.add_rules(tweepy.StreamRule("((Ridwan Kamil) OR (Kang Emil) OR @ridwankamil) lang:in -is:retweet", tag="Ridwan Kamil;tweets"))
client.add_rules(tweepy.StreamRule("((Cak Imin) OR Muhaimin OR @cakimiNOW) lang:in -is:retweet", tag="Muhaimin Iskandar;tweets"))
client.add_rules(tweepy.StreamRule("(Erick Thohir) OR @erickthohir lang:in -is:retweet", tag="Erick Thohir;tweets"))
client.add_rules(tweepy.StreamRule('((Agus (Harimurti OR Yudhoyono)) OR "AHY" OR @AgusYudhoyono) lang:in -is:retweet', tag="Agus Harimurti Yudhoyono;tweets"))
client.add_rules(tweepy.StreamRule('("Airlangga" OR "Hartarto" OR @airlangga_hrt) lang:in -is:retweet', tag="Airlangga Hartarto;tweets"))

client.add_rules(tweepy.StreamRule("(Anies OR @aniesbaswedan) lang:in is:retweet", tag="Anies Baswedan;retweets"))
client.add_rules(tweepy.StreamRule("(Ganjar OR @ganjarpranowo) lang:in is:retweet", tag="Ganjar Pranowo;retweets"))
client.add_rules(tweepy.StreamRule("(Prabowo OR @prabowo) lang:in is:retweet", tag="Prabowo Subianto;retweets"))
client.add_rules(tweepy.StreamRule("(Puan OR @puanmaharani_ri) lang:in is:retweet", tag="Puan Maharani;retweets"))
client.add_rules(tweepy.StreamRule("((Ridwan Kamil) OR (Kang Emil) OR @ridwankamil) lang:in is:retweet", tag="Ridwan Kamil;retweets"))
client.add_rules(tweepy.StreamRule("((Cak Imin) OR Muhaimin OR @cakimiNOW) lang:in is:retweet", tag="Muhaimin Iskandar;retweets"))
client.add_rules(tweepy.StreamRule("(Erick Thohir) OR @erickthohir lang:in is:retweet", tag="Erick Thohir;retweets"))
client.add_rules(tweepy.StreamRule('((Agus (Harimurti OR Yudhoyono)) OR "AHY" OR @AgusYudhoyono) lang:in is:retweet', tag="Agus Harimurti Yudhoyono;retweets"))
client.add_rules(tweepy.StreamRule('("Airlangga" OR "Hartarto" OR @airlangga_hrt) lang:in is:retweet', tag="Airlangga Hartarto;retweets"))

Response(data=[StreamRule(value='("Airlangga" OR "Hartarto" OR @airlangga_hrt) lang:in is:retweet', tag='Airlangga Hartarto;retweets', id='1588041405590822912')], includes={}, errors=[], meta={'sent': '2022-11-03T05:32:30.340Z', 'summary': {'created': 1, 'not_created': 0, 'valid': 1, 'invalid': 0}})

In [5]:
client.get_rules()

Response(data=[StreamRule(value='(Anies OR @aniesbaswedan) lang:in -is:retweet', tag='Anies Baswedan;tweets', id='1588041373718315009'), StreamRule(value='(Ganjar OR @ganjarpranowo) lang:in -is:retweet', tag='Ganjar Pranowo;tweets', id='1588041375744151552'), StreamRule(value='(Prabowo OR @prabowo) lang:in -is:retweet', tag='Prabowo Subianto;tweets', id='1588041377598050304'), StreamRule(value='(Puan OR @puanmaharani_ri) lang:in -is:retweet', tag='Puan Maharani;tweets', id='1588041379409965057'), StreamRule(value='((Ridwan Kamil) OR (Kang Emil) OR @ridwankamil) lang:in -is:retweet', tag='Ridwan Kamil;tweets', id='1588041381276418048'), StreamRule(value='((Cak Imin) OR Muhaimin OR @cakimiNOW) lang:in -is:retweet', tag='Muhaimin Iskandar;tweets', id='1588041383151296512'), StreamRule(value='(Erick Thohir) OR @erickthohir lang:in -is:retweet', tag='Erick Thohir;tweets', id='1588041384896114688'), StreamRule(value='((Agus (Harimurti OR Yudhoyono)) OR "AHY" OR @AgusYudhoyono) lang:in -is:re

In [16]:
class TwitterStreamingClient(tweepy.StreamingClient):
    def __init__(self, bearer_token):
        super().__init__(bearer_token)
        # self.producer = KafkaProducer(
        #     bootstrap_servers=['localhost:9092'],
        #     value_serializer=lambda m: json.dumps(m).encode('utf-8'),
        # )
      
    def on_response(self, response):
        
        data = response.data
        matching_rules = response.matching_rules
        if "users" in response.includes:
            users_dict = {user.id:user for user in response.includes["users"]}
        if "places" in response.includes:
            places_dict = {place.id:place for place in response.includes["places"]}
        if "tweets" in response.includes:
            tweets_dict = {tweet.id:tweet for tweet in response.includes["tweets"]}

        if data is not None:
            tags = [matching_rule.tag.split(';') for matching_rule in matching_rules]
            
            lat, long = None, None
            if data.geo:
                lat1, long1, lat2, long2 = places_dict[data.geo["place_id"]].geo["bbox"]
                lat, long = (lat1+lat2)/2, (long1+long2)/2
            
            message = {
                "id" : data.id,
                "author_username" : users_dict[data.author_id].username,
                "created_at" : data.created_at.isoformat(),
                "text" : data.text,
                "lat" : lat, "long" : long,
                "source" : data.source,
                "retweet_count" : data.public_metrics["retweet_count"],
                "reply_count" : data.public_metrics["reply_count"],
                "like_count" : data.public_metrics["like_count"],
                "quote_count" : data.public_metrics["quote_count"],
                "possibly_sensitive" : data.possibly_sensitive,
                "tags" : [tag[0] for tag in tags],
            }
            
            # self.producer.send(tags[0][1], message)

            print(message)

            # if data.geo:
            #     print(json.dumps(data.data))
            #     print(response.includes)
            #     lat1, long1, lat2, long2 = places_dict[data.geo["place_id"]].geo["bbox"]
            #     print(lat1, long1, lat2, long2)
            #     print(places_dict[data.geo["place_id"]].data)
            #     print("#"*80)

streaming_client_test = TwitterStreamingClient(BEARER_TOKEN)
print(f"Rule used: {streaming_client_test.get_rules()}")

Rule used: Response(data=[StreamRule(value='(Anies OR @aniesbaswedan) lang:in -is:retweet', tag='Anies Baswedan;tweets', id='1588041373718315009'), StreamRule(value='(Ganjar OR @ganjarpranowo) lang:in -is:retweet', tag='Ganjar Pranowo;tweets', id='1588041375744151552'), StreamRule(value='(Prabowo OR @prabowo) lang:in -is:retweet', tag='Prabowo Subianto;tweets', id='1588041377598050304'), StreamRule(value='(Puan OR @puanmaharani_ri) lang:in -is:retweet', tag='Puan Maharani;tweets', id='1588041379409965057'), StreamRule(value='((Ridwan Kamil) OR (Kang Emil) OR @ridwankamil) lang:in -is:retweet', tag='Ridwan Kamil;tweets', id='1588041381276418048'), StreamRule(value='((Cak Imin) OR Muhaimin OR @cakimiNOW) lang:in -is:retweet', tag='Muhaimin Iskandar;tweets', id='1588041383151296512'), StreamRule(value='(Erick Thohir) OR @erickthohir lang:in -is:retweet', tag='Erick Thohir;tweets', id='1588041384896114688'), StreamRule(value='((Agus (Harimurti OR Yudhoyono)) OR "AHY" OR @AgusYudhoyono) lan

In [17]:
streaming_client_test.filter(
    expansions=ALL_EXPANSIONS,
    media_fields=ALL_MEDIA_FIELDS,
    place_fields=ALL_PLACE_FIELDS,
    poll_fields=ALL_POLL_FIELDS,
    tweet_fields=ALL_TWEET_FIELDS,
    user_fields=ALL_USER_FIELDS
)

{'id': 1588053770789060608, 'author_username': 'alisyarief', 'created_at': '2022-11-03T06:21:38+00:00', 'text': 'Kemendagri Kembalikan Permohonan Anies Cabut Pergub Penggusuran Ahok, Kok Bisa? https://t.co/6nw4foYXeD', 'lat': None, 'long': None, 'source': 'Twitter Web App', 'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'possibly_sensitive': False, 'tags': ['Anies Baswedan']}
{'id': 1588053771094937600, 'author_username': 'Faridid4Id', 'created_at': '2022-11-03T06:21:38+00:00', 'text': '@ganjarpranowo Pak monggo kulo aturi pencitraan ten demak pak, sidak jembatan wonokerto, macete mpun mboten kiro", ketambah" patwal mobil pribadi yg tdk ada kepentingannya untuk negara pak', 'lat': None, 'long': None, 'source': 'Twitter for Android', 'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'possibly_sensitive': False, 'tags': ['Ganjar Pranowo']}
{'id': 1588053771854417921, 'author_username': 'haqi11official', 'created_at': '2022-11-03T06:21:38+0

Traceback (most recent call last):
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2204/1654288716.py", line 1, in <module>
    streaming_client_test.filter(
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.10/site-packages/tweepy/streaming.py", line 791, in filter
    if threaded:
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.10/site-packages/tweepy/streaming.py", line 611, in _connect
    def _connect(self, method, endpoint, **kwargs):
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.10/site-packages/tweepy/streaming.py", line 87, in _connect
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.10/site-packages/requests/models.py", line 865, in iter_lines
    for chunk in self.iter_content(
  File "/home/lan666as/miniconda3/envs/nlp-sentiment/lib/python3.1

In [3]:
client.delete_rules([rule.id for rule in client.get_rules().data])

Response(data=None, includes={}, errors=[], meta={'sent': '2022-11-03T05:32:17.379Z', 'summary': {'deleted': 18, 'not_deleted': 0}})